In [ ]:
pip install -U kaleido

In [15]:
import networkx as nx
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as stats
import plotly.express as px
from scipy.stats import entropy
import kaleido
import json
import plotly.io as pio

In [3]:
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive

Mounted at /gdrive
/gdrive


In [102]:
año_inicial = 1999
año_final = 2021

df_resultados_entropia_BTP = pd.read_csv('/gdrive/Shareddrives/GEINCyR - Ciencias Sociales Computacionales Heterodoxas (CSCH)/Carpeta de trabajo/Crisis y robustez en las redes de comercio internacional/Resultados/' + str(año_inicial) + 'to' + str(año_final) + 'resultados_nivel_entropia_BTP.csv')

In [103]:
def dividir_por_maximo(x):
    x['nivel'] = (x['BTP']/x['BTP'].max())#1.0 - x['nivel'] / x['nivel'].max()
    return x

df = df_resultados_entropia_BTP.groupby('año').apply(dividir_por_maximo)

In [104]:
df['BC'] = 1000.0 * df['BC']

In [105]:
# Calcular el promedio de 'BTP' por año
promedio_btp_por_año = df.groupby('año')['BTP'].mean()

# Crear una columna 'BTP_promedio' en el DataFrame original
df['BTP_promedio'] = df['año'].map(promedio_btp_por_año)

# Encontrar las filas con 'BTP' mayores y menores al promedio por año
df_above_avg = df[df['BTP'] > df['BTP_promedio']]
df_below_avg = df[df['BTP'] <= df['BTP_promedio']]

df_above_avg = df_above_avg.drop(['año', 'BTP_promedio'], axis=1)
df_below_avg = df_below_avg.drop(['año', 'BTP_promedio'], axis=1)

df_center_prom = df_above_avg.agg({'nivel': 'mean', 'BTP': 'mean', 'BS': 'mean', 'BC': 'mean', 'BTON': 'mean', 'BUNI': 'mean'}).to_frame().transpose()
df_periphery_prom = df_below_avg.agg({'nivel': 'mean', 'BTP': 'mean', 'BS': 'mean', 'BC': 'mean', 'BTON': 'mean', 'BUNI': 'mean'}).to_frame().transpose()

df_center_std = df_above_avg.agg({'nivel': 'std', 'BTP': 'std', 'BS': 'std', 'BC': 'std', 'BTON': 'std', 'BUNI': 'std'}).to_frame().transpose()
df_periphery_std = df_below_avg.agg({'nivel': 'std', 'BTP': 'std', 'BS': 'std', 'BC': 'std', 'BTON': 'std', 'BUNI': 'std'}).to_frame().transpose()

df_center = pd.concat([df_center_prom, df_center_std], axis=1, keys=['mean', 'std'])
df_center.columns = df_center.columns.map('_'.join)
df_center['Estructura'] = 'Centro'
df_center.set_index('Estructura', inplace=True)

df_periphery = pd.concat([df_periphery_prom, df_periphery_std], axis=1, keys=['mean', 'std'])
df_periphery.columns = df_periphery.columns.map('_'.join)
df_periphery['Estructura'] = 'Periferia'
df_periphery.set_index('Estructura', inplace=True)

df_result = pd.concat([df_center, df_periphery], axis=0)
df_result.reset_index(inplace=True)

In [106]:
df['Estructura'] = df.apply(lambda x: 'Centro' if x['BTP'] >= x['BTP_promedio'] else 'Periferia', axis=1)

In [107]:
año_inicial = 1999
año_final = 2021

df_paises_año = pd.DataFrame()
paises = []

for año in range(año_inicial, año_final + 1):

  with open('/gdrive/Shareddrives/GEINCyR - Ciencias Sociales Computacionales Heterodoxas (CSCH)/Carpeta de trabajo/Crisis y robustez en las redes de comercio internacional/Codigos/estructuras_'+ str(año) +'.json', "r") as f:
    estructuras = json.load(f)

  for nivel in estructuras:
    paises.append(estructuras[nivel]['nodos'])

df['Paises'] = paises

In [108]:
df_result = df_result.rename(columns = {'mean_nivel': 'Gradiente Centro-Periferia', 'mean_BTP': 'Balanza de Poder Comercial Promedio',  'mean_BS': 'Balanza de Entropía [bit] Promedio', 'mean_BC': 'Balanza Comercial [USD] Promedio', 'mean_BTON':'Balanza de Materia [Tm] Promedio', 'mean_BUNI': 'Balanza de Unicidad Promedio',
                                        'std_nivel': 'Gradiente Centro-Periferia Std', 'std_BTP': 'Balanza de Poder Comercial Std',  'std_BS': 'Balanza de Entropía [bit] Std', 'std_BC': 'Balanza Comercial [USD] Std', 'std_BTON':'Balanza de Materia [Tm] Std', 'std_BUNI': 'Balanza de Unicidad Std'})

In [109]:
df = df.rename(columns={'BTP_promedio': 'Balanza de Poder Comercial Promedio por Año', 'nivel': 'Gradiente Centro-Periferia', 'BTP': 'Balanza de Poder Comercial',  'BS': 'Balanza de Entropía [bit]', 'BC': 'Balanza Comercial [USD]', 'BTON':'Balanza de Materia [Tm]', 'BUNI': 'Balanza de Unicidad'})

In [110]:
df.to_csv('/gdrive/Shareddrives/GEINCyR - Ciencias Sociales Computacionales Heterodoxas (CSCH)/Carpeta de trabajo/Crisis y robustez en las redes de comercio internacional/github/' + str(año_inicial) + '_to_' + str(año_final) + '_estructuras_balanzas.csv', index=False)
df_result.to_csv('/gdrive/Shareddrives/GEINCyR - Ciencias Sociales Computacionales Heterodoxas (CSCH)/Carpeta de trabajo/Crisis y robustez en las redes de comercio internacional/github/' + str(año_inicial) + '_to_' + str(año_final) + '_prom_std_balanzas.csv', index=False)

In [111]:
# Establecer un rango mínimo y máximo para los puntos
min_size = 1
max_size = 20

# Normalizar los valores en la columna de tamaño al rango [0, 1]
min_value = df['Balanza de Poder Comercial'].min()
max_value = df['Balanza de Poder Comercial'].max()
df['normalized_size'] = (df['Balanza de Poder Comercial'] - min_value) / (max_value - min_value)

# Ajustar los valores normalizados al rango deseado
df['adjusted_size'] = df['normalized_size'] * (max_size - min_size) + min_size

In [112]:
x, y = 'Balanza de Entropía [bit]', 'Balanza de Materia [Tm]', 
# Calcula el promedio y desviación estándar para cada eje

x_centro = float(df_result[df_result['Estructura'] == 'Centro'][x + ' Promedio'])
x_periferia = float(df_result[df_result['Estructura'] == 'Periferia'][x + ' Promedio'])

x_std_centro = float(df_result[df_result['Estructura'] == 'Centro'][x + ' Std'])
x_std_periferia = float(df_result[df_result['Estructura'] == 'Periferia'][x + ' Std'])

y_centro = float(df_result[df_result['Estructura'] == 'Centro'][y + ' Promedio'])
y_periferia = float(df_result[df_result['Estructura'] == 'Periferia'][y + ' Promedio'])

y_std_centro = float(df_result[df_result['Estructura'] == 'Centro'][y + ' Std'])
y_std_periferia = float(df_result[df_result['Estructura'] == 'Periferia'][y + ' Std'])

In [113]:
fig = px.scatter(df, x=x, y=y, color='Gradiente Centro-Periferia', color_continuous_scale=["rgb(0, 0, 204)", "rgb(204, 0, 0)"], size='adjusted_size', size_max=max_size,
                  template="ggplot2") #,title= x + " - " + y 
fig.update_layout(
    plot_bgcolor='rgb(255, 255, 255)', 
    width=800,
    height=600,
    font=dict(
        family='Arial, sans-serif',
        size=18,
        color='black'
    ),
    xaxis=dict(exponentformat = "power",
        title=dict(
            font=dict(
                family='Arial, sans-serif',
                size=25,
                color='black'
            )
        ),
        tickfont=dict(
            family='Arial, sans-serif',
            size=16,
            color='black'
        )
    ),
    yaxis=dict(exponentformat = "power",
        title=dict(
            font=dict(
                family='Arial, sans-serif',
                size=25,
                color='black'
            )
        ),
        tickfont=dict(
            family='Arial, sans-serif',
            size=16,
            color='black'
        )
    ),
    coloraxis_colorbar=dict(
        title_side="right",
        tickmode='array',
        tickvals=[.0,.2,.4,.6,.8,1.0],
        title=dict(                 
            font=dict(
                family='Arial, sans-serif',
                size=25,
                color='black'
            ),
        ),
        tickfont=dict(
            family='Arial, sans-serif',
            size=16,
            color='black'
        ),
        thicknessmode='pixels',
        thickness=15,         
        y=0.5,
        x=1.0
    )
)


fig.update_xaxes(showgrid=True, showline=True, linewidth=2, linecolor='black', mirror=True)
fig.update_yaxes(showgrid=True, showline=True, linewidth=2, linecolor='black', mirror=True)

# Agrega las líneas verticales y horizontales a la gráfica

fig.add_hline(y=y_centro, line_width=3.5, line_color="red", opacity=0.9)
fig.add_hline(y=y_periferia, line_width=3.5, line_dash="dash", line_color="blue", opacity=0.9)

fig.add_hrect(y0=y_centro - y_std_centro, y1=y_centro + y_std_centro, 
              annotation_text="", annotation_position="top left",
              fillcolor="red", opacity=0.12, line_width=0)
fig.add_hrect(y0=y_periferia - y_std_periferia, y1=y_periferia + y_std_periferia, 
              annotation_text="", annotation_position="top left",
              fillcolor="blue", opacity=0.12, line_width=0)

#fig.update_traces(marker_size=10)
fig.add_vline(x=0.0, line_width=2, line_color="black", opacity=0.9)
fig.add_hline(y=0.0, line_width=2, line_color="black", opacity=0.9)
fig.show()

#pio.write_image(fig,"/gdrive/Shareddrives/GEINCyR - Ciencias Sociales Computacionales Heterodoxas (CSCH)/Carpeta de trabajo/Crisis y robustez en las redes de comercio internacional/graficas/"+ x +"_vs_"+ y+".png")
